In [7]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib import rcParams
import lmfit
from lmfit import minimize, Parameters
import time
import math

%matplotlib inline

# Load the covariances

In [2]:
RotatedCovariance = np.loadtxt('../Rotation-2pt/rotation-tweaks/RotParameters/cov_c.txt', dtype=np.float64)
RotatedCovariance = RotatedCovariance[:16,:16]
RotatedGaussianCovariance = np.loadtxt('../BJY1/ParametersBJ/cov_c.txt', dtype=np.float64)
RotatedGaussianCovariance = RotatedGaussianCovariance[:16,:16]

This doesn't seem to run properly for some reason, so let's run it separately.

In [4]:
def Correlation_from_covariance(covmat):
    v = np.sqrt(np.diag(covmat))
    outer_v = np.outer(v, v)
    correlation = covmat / outer_v
    correlation[covmat == 0] = 0
    return correlation

# Cool, awesome code here

In [5]:
class FindError(object):
    
    def __init__(self, testing_covmat, base_covmat, which=None, where=None, sample_size=1000, method='powell'):

        self.testing_covmat = testing_covmat
        self.base_covmat = base_covmat
        self.sample_size = sample_size
        self.base_corr, self.testing_corr = self.get_corr()
        self.where = where
        self.error = {}
        #self.which = which
        params = lmfit.Parameters()
        params.add('err', value=np.pi, min=0.01, max=20)
        
        self.which = "diag"
        self.base, self.testing = self.get_stuff()
        self.diag_results = self.get_bounds(params, method)
        self.error[self.which] = self.diag_results
        self.which = "corr"
        self.base, self.testing = self.get_stuff()
        self.corr_results = self.get_bounds(params, method)
        self.error[self.which] = self.corr_results
    
    def get_stuff(self):
        if self.which == "corr":
            base = self.base_corr
            testing = self.testing_corr
        else:
            base = np.diag(self.base_covmat)
            testing = np.diag(self.testing_covmat)
        return base, testing
        
    def Correlation_from_covariance(covmat):
        v = np.sqrt(np.diag(covmat))
        outer_v = np.outer(v, v)
        correlation = covmat / outer_v
        correlation[covmat == 0] = 0
        return correlation
    
    def get_chi2(self,sample1,sample2):
        diff = np.ravel(sample1-sample2)
        part = diff@(self.invS)
        return part@diff.T
    
    def get_corr(self):
        length = len(self.testing_covmat)
        a = length*(length+1)/2 - length
        corr_b = np.zeros(int(a))
        corr_t = np.zeros(int(a))
        n = 0
        start = 1
        for i in range(length):
            for j in range(start, length):
                corr_b[n] = Correlation_from_covariance(self.base_covmat)[i,j]
                corr_t[n] = Correlation_from_covariance(self.testing_covmat)[i,j]
                n += 1
            start += 1
        return corr_b, corr_t    
    
    def get_perturbed(self, params):
        err = params['err'].value
        if len(self.base) == len(self.base_covmat):
            diag_sqrt = np.sqrt(self.base)
            for j in range(len(diag_sqrt)):
                a = np.random.normal(0,err/100)
                diag_sqrt[j] = diag_sqrt[j]*(1+a)
            diag_new = (diag_sqrt)**2
            return diag_new
        else:
            corr_new = []
            for i in range(len(self.base)):
                y = math.atanh(self.base[i])
                dyin = np.random.normal(0,err/100)
                dy = dyin*(math.cosh(y + dyin/2))**2
                new_val = y+dy
                corr_new.append(math.tanh(new_val))
            return corr_new
        
    def get_distChi2(self, params):
        mocks = np.zeros((self.sample_size, len(self.base)))
        for s in range(self.sample_size):
            mocks[s] = self.get_perturbed(params)
        self.invS = np.linalg.inv(np.cov(mocks.T))
        mean = []
        for i in range(len(self.base)):
            mean.append(np.mean(mocks.T[i,:]))
        chi2 = []
        for i in range(self.sample_size):
            chi2.append(self.get_chi2(mocks[i], mean))
        return np.array(chi2), mean, self.invS   
    
    def get_toleranceBounds(self, params):
        sample, mean, self.invS = self.get_distChi2(params)
        if self.where=="lower":
            confidence_interval = np.std(sample)
            lower_val = np.mean(sample) + confidence_interval
            return (self.get_chi2(mean, self.testing) - lower_val)**2
        #if self.where=="upper":
        #    confidence_interval = np.std(sample)
        #    upper_val = np.mean(sample) + confidence_interval
        #    return (self.get_chi2(mean, self.testing) + upper_val)**2
        else:
            return (self.get_chi2(mean, self.testing) - np.mean(sample))**2
    
    def get_bounds(self, params, method):
        self.where = "lower"
        self.sample_size = int(self.sample_size/2)
        get_summary = lmfit.minimize(self.get_toleranceBounds, params, args=(), method=method)
        lower_bound = get_summary.params.get('err').value
        #self.where = "upper"
        #self.sample_size = int(self.sample_size/2)
        #get_summary = lmfit.minimize(self.get_toleranceBounds, params, args=(), method=method)
        #upper_bound = get_summary.params.get('err').value
        self.where = "bf"
        self.sample_size = int(self.sample_size*2)
        get_summary = lmfit.minimize(self.get_toleranceBounds, params, args=(), method=method)
        bf = get_summary.params.get('err').value
        interval = abs(bf - lower_bound)
        #interval = abs(upper_bound - lower_bound)/2
        return np.round(bf,1), np.round(interval,1)

In [ ]:
start_time = time.time()
testing = FindError(RotatedGaussianCovariance, RotatedCovariance, sample_size=1000)
print(testing.error)
print(round(time.time() - start_time, 3), 'seconds or', round((time.time() - start_time)/60, 3), 'minutes')